### Домашнее задание №5 к лекции "Основы веб-скрапинга"

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
from datetime import datetime
import re
from tqdm import tqdm

#### Задание 1
Вам необходимо написать функцию, которая будет основана на поиске по сайту http://habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>
В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [2]:
req = ['большие данные', 'python', 'sql', 'big data']

Так как HTML извлекать придется во каждом случае, создадим для этого вспомогательную функцию.

In [3]:
def get_html(url, params=None):
    ''' 
    Функция для получения HTML кода из url в случае 200 ответа сайта.
    В противном - ничего
    '''
    html = requests.get(url, params=params)
    if html.status_code == 200:
        return html.text

In [4]:
def get_content_lite(req):
    '''
    Функия получения короткой таблицы со статьями по списку поисковых слов
    На вход принимает список слов.
    Внутри:
    1) для каждого слова делает запрос к сайту
    2) перебирает блоки со статьями на первой странице поиска
    3) для каждой статьи собирает дату, название, ссылку, автора
    4) добавляет конкретное поисковое слово
    5) доавляет полученную строку к датафрейму
    6) проходит в цикле по всем словам из переданного списка
    7) удаляет дубли по столбцу со ссылками
    8) работает только для первой страницы поиска
    '''
    url = 'https://habr.com/ru/search/'
    df = pd.DataFrame()
    for word in req:
        params = {'q': word}
        html = get_html(url, params=params) # используем вспомогательную функцию для получения html
        soup = BeautifulSoup(html, 'html.parser')
        articles = soup.find('div', class_='tm-articles-list').find_all('div', 'tm-article-snippet')
        for item in tqdm(articles):
            sleep(0.1)
            date = pd.to_datetime(item.find('time').get('datetime')).strftime('%Y-%m-%d %H:%M') 
            title = item.find('a', 'tm-article-snippet__title-link').text
            link = 'https://habr.com' + item.find('a', 'tm-article-snippet__title-link').get('href')
            author = item.find('span', 'tm-user-info__user').text.strip()
            row = {'word':word, # собираем строку
                   'date':date, 
                   'title':title, 
                   'link':link, 
                   'author':author}
            df = df.append(row, ignore_index=True) # добавляем строку
    df.drop_duplicates(subset='link', inplace=True) # удаляем дубли
    return df

In [5]:
%%time
content_lite = get_content_lite(req)

100%|██████████| 19/19 [00:02<00:00,  8.85it/s]

CPU times: user 1.15 s, sys: 102 ms, total: 1.26 s
Wall time: 12.3 s


In [6]:
content_lite.sort_values(by='date', ascending=False).head(10)

,word,date,title,link,author
35,python,2022-05-18 15:33,Слёрм запускает 3-дневный интенсив по Python д...,https://habr.com/ru/company/southbridge/news/t...,edeshina
37,python,2022-04-22 11:42,TechnoMeetsPython. Онлайн митап о Python-разра...,https://habr.com/ru/news/t/662437/,technokratiya
13,большие данные,2022-04-07 14:23,17 лучших инструментов и технологий для работы...,https://habr.com/ru/company/otus/blog/659657/,kmoseenk
40,sql,2022-04-07 08:56,Яндекс Практикум запускает курс «SQL для работ...,https://habr.com/ru/company/yandex_praktikum/n...,eshulyndina
36,python,2022-03-18 15:31,24 марта Слёрм проведёт открытый урок «Первый ...,https://habr.com/ru/company/southbridge/news/t...,edeshina
62,big data,2022-03-10 08:30,10—24 марта: Big Data Dev Week от билайна,https://habr.com/ru/company/beeline/news/t/654...,Bee_brightside
25,python,2022-03-08 09:13,Вышел мартовский релиз расширения Python для V...,https://habr.com/ru/news/t/654707/,maybe_elf
74,big data,2022-02-18 12:51,Citymobil Data Meetup №7,https://habr.com/ru/company/citymobil/news/t/6...,leleles
19,python,2022-01-20 15:37,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/company/southbridge/news/t...,Hedgehog_art
63,big data,2022-01-20 11:02,Citymobil Data Meetup №6,https://habr.com/ru/company/citymobil/news/t/6...,leleles


Для каждого поискового слова скрипт срабатывает за 2 секунды

#### Задание 2  
Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [7]:
req = ['python', 'mlflow', 'airflow', 'sql']

Добавим еще две вспомогательные функции.  

1) Для определения количества страниц поиска (это максимальное количество, дальше поиск работать не будет)  
2) Для получения полного текста статьи и количества оценок

In [8]:
def get_num_pagins(url, word):
    '''Фукнция принимает на вход url и поисковое слово.
    Переходит на первую страницу поиска и 
    возвращает количество страниц
    '''
    params = {'q': word}
    html = get_html(url, params=params)
    soup = BeautifulSoup(html, 'html.parser')
    num = int(re.findall(r'\d+', soup.find_all('div', 'tm-pagination__page-group')[-1].text)[-1])
    return num

In [9]:
for word in req:
    print(word, get_num_pagins('https://habr.com/ru/search/', word))

python 50
mlflow 3
airflow 11
sql 50


Похоже 50 - максимальное количество страниц, для некоторых слов станиц меньше.  
Особенность в том, что если задать в запросе номер страницы превышающий максимум ответ будет __"Страница не найдена"__ и скрипт сломается. Это ограничение необходимо предусмотреть в итоговой функции.

In [10]:
def get_article_details(url):
    '''
    Фунция принимает на вход url конкретной статьи.
    Возвращает кортеж из двух значений:
    - число баллов (может быть и отрицательным)
    - текст новости, очищенный от пробельных символов
    '''
    html = get_html(url)
    soup = BeautifulSoup(html, 'html.parser')
    votes = soup.find('div', class_="tm-votes-meter tm-article-rating__votes-switcher")
    content = soup.find('div', class_='tm-article-body')
    if votes is not None: # в некоторых статьях по этим тегам может ничего не вернуться
        votes = int((votes).find('span').text) 
    if content is not None: # в некоторых статьях по этим тегам может ничего не вернуться
        content = re.sub(r'\s+',' ', content.get_text().strip()) # очищаем текст от переносов и табуляций
    return (votes, content)

In [11]:
%%time
get_article_details('https://habr.com/ru/news/t/497474/')

CPU times: user 48.8 ms, sys: 3.6 ms, total: 52.4 ms
Wall time: 252 ms


(13,
 'В рамках развития программы Microsoft AI for Earth в компании анонсировали новые этапы информационно-технические этапы по сохранению биоразнообразия и природных экосистем нашей планеты. 15 апреля 2020 года Microsoft объявила, что скоро запустит открытую вычислительную платформу под названием «Планетарный компьютер» для сбора, хранения и анализа данных о состоянии Земли. Причем доступ к платформе как для загрузки данных, так и для получения информации о состоянии Земли, например, изменении размеров лесных массивов, оценки рисков затоплений, землетрясений и других природных катастроф, бесплатно получат исследователи, экологи, ученые, специалисты по охране природы и окружающего мира, некоммерческие организации и государственные учреждения всего мира. Microsoft AI for Earth — это часть глобального проекта компании под названием AI for Good, направленного на применение технологий искусственного интеллекта для борьбы с тремя глобальными проблемами: загрязнением окружающей среды (AI fo

Читать статьи в таком формате не очень удобно. Извлеченный текст скорее подойдет для лингвистического анализа, векторизации, частоты упоминаний различных инструментов и терминов и т.п.

In [12]:
def get_content_pro(req, pages=4):
    '''
    Функия получения полной таблицы со статьями по списку поисковых слов
    На вход принимает список слов и число станиц поиска
    Внутри:
    1) для каждого слова делает запрос к сайту
    2) перебирает блоки со статьями на первой странице поиска
    3) для каждой статьи собирает дату, название, ссылку, атора
    4) переходит по сстылке статьи и забирает оттуда оценки и полный текст
    5) добавляет конкретное поисковое слово
    6) доавляет полученную строку к датафрейму
    7) проходит по всем страницам, но не более чем передано в функцию
    8) проходит в цикле по всем словам из переданного списка
    9) удаляет дубли по столбцу со ссылками
    '''
    url = 'https://habr.com/ru/search/'
    df = pd.DataFrame()
    for word in req:
        params = {'q': word}
        current_word_pages = int(pages) # для текущего слова задаем кол-во страниц из функции
        max_pages = get_num_pagins(url, word) # используем вспомогательную функцию для получения максимальных страниц
        if current_word_pages > max_pages: # если кол-во страниц превышает максимальное 
            current_word_pages = max_pages # приравниваем к максимуму
        else:
            current_word_pages # в противном случае - сколько задали изначально
        for page in range(1, current_word_pages+1):
            html = get_html(url+f'page{page}/', params=params) # используем вспомогательную функцию для получения html
            soup = BeautifulSoup(html, 'html.parser')
            articles = soup.find('div', class_='tm-articles-list').find_all('div', 'tm-article-snippet')
            for item in tqdm(articles):
                sleep(0.1)
                date = pd.to_datetime(item.find('time').get('datetime')).strftime('%Y-%m-%d %H:%M') 
                title = item.find('a', 'tm-article-snippet__title-link').text
                link = 'https://habr.com' + item.find('a', 'tm-article-snippet__title-link').get('href')
                votes = get_article_details(link)[0] # используем вспомогательную функцию для получения голосов
                text = get_article_details(link)[1] # используем вспомогательную функцию для получения всего текста
                author = item.find('span', 'tm-user-info__user').text.strip()
                row = {'word':word, # собираем строку
                       'date':date, 
                       'title':title, 
                       'link':link, 
                       'author':author, 
                       'votes':votes,
                       'text':text}
                df = df.append(row, ignore_index=True) # добавляем строку
    df.drop_duplicates(subset='link', inplace=True) # удаляем дубли
    return df

In [13]:
%%time
content_pro = get_content_pro(req, 5)

100%|██████████| 20/20 [00:20<00:00,  1.03s/it]

CPU times: user 54.8 s, sys: 1.58 s, total: 56.4 s
Wall time: 5min 47s


In [14]:
content_pro.sort_values(by='date', ascending=False).head(10)

,word,date,title,link,author,votes,text
22,python,2022-05-24 07:57,Дайджест Слёрма: тест на уровень кунг-фу по Py...,https://habr.com/ru/company/southbridge/news/t...,Lika_Chernigo,9.0,Сделали для вас подборку свежих статей и выгод...
16,python,2022-05-18 15:33,Слёрм запускает 3-дневный интенсив по Python д...,https://habr.com/ru/company/southbridge/news/t...,edeshina,5.0,24-26 июня пройдёт онлайн-интенсив для инженер...
204,airflow,2022-05-13 17:03,Кто такой Analytics Engineer – E2E-решение с и...,https://habr.com/ru/company/otus/blog/665642/,kzzzr,7.0,"Привет! Меня зовут Артемий Козырь, и я Analyti..."
150,mlflow,2022-05-12 06:56,Как и для чего мы построили ML Space,https://habr.com/ru/company/sbercloud/blog/665...,SberCloud_Administrator,10.0,Речь пойдет о платформе для ML-разработки полн...
209,airflow,2022-05-05 06:14,"«Божественная комедия», или Девять кругов прог...",https://habr.com/ru/company/magnit/blog/664358/,He6puToCTb,8.0,"Привет, Хабр! На связи команда направления про..."
149,mlflow,2022-05-04 08:44,Data-Science-процессы: Jupyter Notebook для пр...,https://habr.com/ru/company/vk/blog/662734/,Olga_Mokshina,37.0,Jovian Blues by ShootingStarLogBook Рефакторин...
197,airflow,2022-04-25 08:43,Сравнение процессов ETL и ELT,https://habr.com/ru/post/662746/,LiMalk,0.0,"ETL означает извлечение, преобразование и загр..."
106,mlflow,2022-04-23 05:48,Почему инструменты MLOps должны быть с открыты...,https://habr.com/ru/post/662519/,mnrozhkov,1.0,Перевод статьи подготовлен совместно с Моргуно...
18,python,2022-04-22 11:42,TechnoMeetsPython. Онлайн митап о Python-разра...,https://habr.com/ru/news/t/662437/,technokratiya,2.0,27 апреля в 18:00 собираем питонистов на YouTu...
232,airflow,2022-04-20 08:07,"Четыре хитрости в работе с пайплайнами данных,...",https://habr.com/ru/company/vk/blog/659389/,Olga_Mokshina,13.0,Dust-n-Rust by Spiritofdarkness Команда разраб...


Для одного слова на страницу требуется примерно 17 секунд. По замыслу скрипт стоит обернуть в питоновский файл.py и запускать по расписанию ночью. Все 50 страниц поиска должны выгрузиться за примерно 15-16 мин, следовательно 4 слова за 1 час, следовательно за 4 часа можно обработать 16 слов. Если понадобится больше слов, можно ограничиться 4-5 страницами.

In [15]:
content_pro.groupby('word').agg({'link':'count'})

,link
word,
airflow,92
mlflow,51
python,100
sql,100


In [16]:
req_2 = ['airflow']

In [17]:
%%time
content_pro_airflow = get_content_pro(req_2, 5)
content_pro_airflow.groupby('word').agg({'link':'count'})

100%|██████████| 20/20 [00:18<00:00,  1.07it/s]

CPU times: user 14.4 s, sys: 482 ms, total: 14.9 s
Wall time: 1min 29s


,link
word,
airflow,99


Все логично.  
- У слов __python__ и __sql__ собрано по 5 страниц с 20 статьями - всего по 100. 
- У слова __mlflow__ 3 страницы 20+20+11=51 статья
- У слова __airflow__ тоже 5 страниц. всего 99 статей, но 7 удалились как дубли в смежных запросах